# Startegi : Add more layer in neural network predict with output 3 dense

In [1]:
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

Using TensorFlow backend.


In [6]:
path='data/train.csv'

In [7]:
df=pd.read_csv(path, index_col=0)
df=df[['x','y','z','x_sim','y_sim','z_sim']]
# df=df.drop('epoch', axis=1)
df.head()

,x,y,z,x_sim,y_sim,z_sim
id,,,,,,
0,-8855.823863,13117.780146,-20728.353233,-8843.131454,13138.221690,-20741.615306
1,-10567.672384,1619.746066,-24451.813271,-10555.500066,1649.289367,-24473.089556
2,-10578.684043,-10180.467460,-24238.280949,-10571.858472,-10145.939908,-24271.169776
3,-9148.251857,-20651.437460,-20720.381279,-9149.620794,-20618.200201,-20765.019094
4,-6719.092336,-28929.061629,-14938.907967,-6729.358857,-28902.271436,-14992.399986


In [8]:
mydata=df.copy()

In [9]:
mydata.isna().sum()

x        0
y        0
z        0
x_sim    0
y_sim    0
z_sim    0
dtype: int64

In [10]:
mydata = mydata.dropna()
mydata.shape

(649912, 6)

In [11]:
train_mydata = mydata.sample(frac=0.8,random_state=0)
test_mydata = mydata.drop(train_mydata.index)
test_mydata.head()

,x,y,z,x_sim,y_sim,z_sim
id,,,,,,
1,-10567.672384,1619.746066,-24451.813271,-10555.500066,1649.289367,-24473.089556
4,-6719.092336,-28929.061629,-14938.907967,-6729.358857,-28902.271436,-14992.399986
15,17281.246143,-2011.121378,39970.739873,17241.395285,-2016.770194,39958.759155
22,-1447.724638,28032.228987,-3918.192838,-1461.093061,28026.085525,-3959.767549
26,-10583.521489,-10145.003880,-24243.775722,-10574.016380,-10147.909472,-24270.149411


In [12]:
#a=train_mydata.shape
#b=test_mydata.shape
#print(a,b)
print(train_mydata.shape, test_mydata.shape)
train_mydata.head()

(519930, 6) (129982, 6)


,x,y,z,x_sim,y_sim,z_sim
id,,,,,,
162906,-22445.306373,5848.139687,805.906588,-21786.206588,7375.036638,391.178832
123662,-9086.781838,7654.307240,-6721.299832,-9545.479053,7482.025507,-6387.708079
901720,-33475.100342,-37591.455624,-3269.507831,-33263.131053,-37783.198553,-3617.165346
308669,11617.034853,2343.826977,-1821.565305,10718.558812,5174.769686,-1069.308627
1154759,25113.931453,-473.679010,41867.498593,19229.123354,12147.342145,31835.248429


In [13]:
label=['Vx','Vy','Vz','x','y','z'][3:]
train_stats = train_mydata.describe()
for i in label:
  train_stats.pop(i)
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
x_sim,519930.0,-941.331006,25387.037253,-255037.952966,-13139.771199,-612.877803,11194.372299,200961.158000
y_sim,519930.0,319.853163,25603.245579,-281784.430040,-11758.892200,501.609913,12558.620911,209619.553857
z_sim,519930.0,-77.075471,20670.833347,-180250.922287,-7963.461284,57.093264,8253.028193,198526.091552


In [14]:
train_labels = np.array([i for i in train_mydata[label].values])
test_labels = np.array([i for i in test_mydata[label].values])

In [15]:
print(train_labels[0], test_labels[0])

[-22445.30637347   5848.13968719    805.90658772] [-10567.67238391   1619.74606635 -24451.81327102]


In [16]:
def norm(x):
  try:
    return (x - train_stats['mean']) / train_stats['std']
  except:
    print(x)

In [17]:
normed_train_data = norm(train_mydata.drop(label, axis=1))
normed_test_data = norm(test_mydata.drop(label, axis=1))

In [18]:
a=normed_test_data.describe()
a.transpose()

,count,mean,std,min,25%,50%,75%,max
x_sim,129982.0,-0.001650,0.997447,-10.002493,-0.483600,0.011332,0.477304,7.940276
y_sim,129982.0,0.000919,1.000829,-11.059974,-0.467484,0.011119,0.476295,8.154777
z_sim,129982.0,-0.000651,0.999315,-8.628536,-0.380103,0.006436,0.399140,9.581585


In [21]:
normed_test_data.shape

(129982, 3)

In [25]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = normed_test_data.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(3, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               512       
_________________________________________________________________
dense_7 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_8 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_9 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 771       
Total params: 165,891
Trainable params: 165,891
Non-trainable params: 0
_________________________________________________________________


In [26]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [27]:
NN_model.fit(normed_train_data, train_labels, validation_data=(normed_test_data, test_labels), epochs=10)

Train on 519930 samples, validate on 129982 samples
Epoch 1/10
519930/519930 [==============================] - 43s 82us/step - loss: 3436.6121 - mean_absolute_error: 3436.6038 - val_loss: 2959.7068 - val_mean_absolute_error: 2959.7019
Epoch 2/10
519930/519930 [==============================] - 44s 85us/step - loss: 2931.0616 - mean_absolute_error: 2931.0613 - val_loss: 2913.9964 - val_mean_absolute_error: 2913.9971
Epoch 3/10
519930/519930 [==============================] - 44s 84us/step - loss: 2922.8698 - mean_absolute_error: 2922.8672 - val_loss: 2999.9013 - val_mean_absolute_error: 2999.9011
Epoch 4/10
519930/519930 [==============================] - 44s 84us/step - loss: 2918.6911 - mean_absolute_error: 2918.6892 - val_loss: 2935.1121 - val_mean_absolute_error: 2935.1094
Epoch 5/10
519930/519930 [==============================] - 45s 86us/step - loss: 2912.2598 - mean_absolute_error: 2912.2566 - val_loss: 2892.4020 - val_mean_absolute_error: 2892.4065
Epoch 6/10
519930/519930 [==

In [32]:
test_predictions = NN_model.predict(normed_test_data)

In [33]:
test_predictions

array([[-10004.26   ,   1734.4694 , -24788.012  ],
       [ -6490.606  , -28718.744  , -15219.35   ],
       [ 16965.803  ,  -2342.4812 ,  39375.793  ],
       ...,
       [  9192.397  , -26044.5    ,     96.09594],
       [  4642.833  , -27569.176  ,   1064.1875 ],
       [-23356.35   ,  -5476.5938 ,   4563.947  ]], dtype=float32)

In [34]:
test_labels

array([[-10567.67238391,   1619.74606635, -24451.81327102],
       [ -6719.09233577, -28929.06162891, -14938.90796702],
       [ 17281.24614306,  -2011.12137796,  39970.73987322],
       ...,
       [  6809.32472235, -27094.76801954,    743.39273636],
       [  2071.20677085, -28307.11297851,   1753.73003927],
       [-23363.04479387,  -2906.07132041,   4747.2473861 ]])

In [35]:
def smape(satellite_predicted_values, satellite_true_values):
    return np.mean(np.abs((satellite_predicted_values - satellite_true_values)/(np.abs(satellite_predicted_values) + np.abs(satellite_true_values))))

In [43]:
test_predictions = NN_model.predict(normed_test_data)
sim=test_mydata[[i+'_sim' for i in label]].values
print('smape real vs pred : %f'%smape(test_predictions, test_labels))
print('smape real vs sim : %f'%smape(sim, test_labels))

smape real vs pred : 0.190534
smape real vs sim : 0.184052
